In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from multiprocessing import Pool, Process
from haversine import haversine
import os
import json
from collections import OrderedDict

In [5]:
df = pd.read_csv('./Data/test_link1.txt')

In [6]:
df

,lon,lat,index,point_in_time,od_id,seq,flink,tlink,turn,flink_len,...,track_in_date,track_in_time,track_speed,track_travel_time,track_sum_travel_time,static_spd,hq_speed,ol_rot_spd,ol_spd,svc_name
0,127.062528,37.510028,3,110855,20190421B03286079,61,2033907,2065408,3,1190,...,20190421,110851,432,99,7235,44,0,56,56,0
1,127.062417,37.510194,4,110857,20190421B03286079,61,2033907,2065408,3,1190,...,20190421,110851,432,99,7235,44,0,56,56,0
2,127.062333,37.510361,5,110859,20190421B03286079,61,2033907,2065408,3,1190,...,20190421,110851,432,99,7235,44,0,56,56,0
3,127.062250,37.510500,6,110900,20190421B03286079,61,2033907,2065408,3,1190,...,20190421,110851,432,99,7235,44,0,56,56,0
4,127.062250,37.510500,0,110901,20190421B03286079,62,2065408,2065411,3,770,...,20190421,110901,525,53,7288,52,0,60,60,0
5,127.062167,37.510667,1,110902,20190421B03286079,62,2065408,2065411,3,770,...,20190421,110901,525,53,7288,52,0,60,60,0
6,127.062083,37.510833,2,110903,20190421B03286079,62,2065408,2065411,3,770,...,20190421,110901,525,53,7288,52,0,60,60,0
7,127.061972,37.511000,3,110904,20190421B03286079,62,2065408,2065411,3,770,...,20190421,110901,525,53,7288,52,0,60,60,0
8,127.061917,37.511139,4,110906,20190421B03286079,62,2065408,2065411,3,770,...,20190421,110901,525,53,7288,52,0,60,60,0
9,127.061917,37.511139,0,110906,20190421B03286079,63,2065411,2065406,3,1500,...,20190421,110906,633,85,7373,49,0,69,69,0


In [7]:
df = df[['lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
             'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
             'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
             'ol_rot_spd', 'ol_spd', 'svc_name', 'od_id']]

In [8]:
df

,lon,lat,index,point_in_time,seq,flink,tlink,turn,flink_len,sum_len,...,track_in_time,track_speed,track_travel_time,track_sum_travel_time,static_spd,hq_speed,ol_rot_spd,ol_spd,svc_name,od_id
0,127.062528,37.510028,3,110855,61,2033907,2065408,3,1190,58250,...,110851,432,99,7235,44,0,56,56,0,20190421B03286079
1,127.062417,37.510194,4,110857,61,2033907,2065408,3,1190,58250,...,110851,432,99,7235,44,0,56,56,0,20190421B03286079
2,127.062333,37.510361,5,110859,61,2033907,2065408,3,1190,58250,...,110851,432,99,7235,44,0,56,56,0,20190421B03286079
3,127.062250,37.510500,6,110900,61,2033907,2065408,3,1190,58250,...,110851,432,99,7235,44,0,56,56,0,20190421B03286079
4,127.062250,37.510500,0,110901,62,2065408,2065411,3,770,59020,...,110901,525,53,7288,52,0,60,60,0,20190421B03286079
5,127.062167,37.510667,1,110902,62,2065408,2065411,3,770,59020,...,110901,525,53,7288,52,0,60,60,0,20190421B03286079
6,127.062083,37.510833,2,110903,62,2065408,2065411,3,770,59020,...,110901,525,53,7288,52,0,60,60,0,20190421B03286079
7,127.061972,37.511000,3,110904,62,2065408,2065411,3,770,59020,...,110901,525,53,7288,52,0,60,60,0,20190421B03286079
8,127.061917,37.511139,4,110906,62,2065408,2065411,3,770,59020,...,110901,525,53,7288,52,0,60,60,0,20190421B03286079
9,127.061917,37.511139,0,110906,63,2065411,2065406,3,1500,60520,...,110906,633,85,7373,49,0,69,69,0,20190421B03286079


In [9]:
first_index = df[(df['index']== 0) & (df['seq'] == 0)].index
first_index

Int64Index([63, 514, 823, 1164, 1380, 2982, 3425, 3668, 3798, 4190, 4387, 4747,
            4830],
           dtype='int64')

In [10]:
def Make_id_parser(data):
    if not os.path.isdir('./Data/id_based_parsing_22'):
        os.mkdir('./Data/id_based_parsing_22')
    for i in range (len(data)):
        id_in = df.loc[data[i], 'od_id']
        if(i != len(data)-1):
            file_out = df.loc[data[i]:(data[i+1]-1)]
            file_out.to_csv('./Data/id_based_parsing_22/id_{}.csv'.format(id_in))

In [11]:
Make_id_parser(first_index) #start make id_based_parsing data

In [12]:
def Make_preprocess(folder_name, start, end, num_file, IDs, calc_time,divide_num):
    # make directory
    if not os.path.isdir('./Data/'+folder_name):
        os.mkdir('./Data/'+folder_name)
    
    f =open("./Data/"+folder_name+"/data_{:02d}".format(num_file+1), 'a')
    for count in range(start, end):
        
    #if not exist file, skip
        if not os.path.isfile('./Data/id_based_parsing_22/id_{}.csv'.format(IDs[count])):
            print('id_{}.csv can not be found'.format(IDs[count]))
            count = count + 1
            continue
            
    #read id file            
        df1 = pd.read_csv('./Data/id_based_parsing_22/id_{}.csv'.format(IDs[count]))
        ## 링크의 끝점 보단 링크의 시작점을 인덱싱하는 것이 좋겠다고 판단하여 수정
        first_index = df1[(df1['index']== 0)].index
        #last_index = np.delete(last_index,0)
        first_index = np.append(first_index,len(df1.index)-1)

        index = [0] ###경로의 시작점을 명시하기 위해 수정
        j=0
                    
        for i, last in enumerate(first_index):  ## divide_num = 5 (100m)
            while ( j != last ) :
                if ( j+ divide_num > last ):
                    if ( i != 0 ):
                        j = j + (first_index[i]-first_index[i-1])%divide_num
                    else :
                        j = j = j + (first_index[i]-0)%divide_num

                    index.append(j)
                else :
                    j = j+divide_num
                    index.append(j)

                if ( j == last ):
                    break

        df1 = df1.loc[index]
        df1 = df1.reset_index()
        df2 = df1.loc[index[0]]
         
    
    # lngs, lats
        df2['lngs'] = df1['lon'].values
        df2['lats'] = df1['lat'].values
        

    # time, dist
        tot_time = df1.loc[df1.index[-1], 'track_sum_travel_time']
        tot_dist = df1.loc[df1.index[-1], 'sum_len']
        df2['time'] = tot_time / 10
        df2['dist'] = tot_dist / 10000
        

    # to datetime
        time = df1.loc[index[0], 'track_in_time']
        date = df1.loc[index[0], "track_in_date"]
        test = str(date) + str('%6d' %time)
        test = test.replace(' ', '0')
        test = pd.to_datetime(test)
        
    # weekID, dateID, timeID
        weekID = test.weekday()
        df2['weekID'] = weekID
        df2['dateID'] = test.day
        timeID = (test.hour * 60) + (test.minute) + int(test.second / 60)
        df2['timeID'] = timeID
        
    # driverID
        df2 = df2.rename({'od_id': 'driverID'})
        driver =df2['driverID']
        driver = driver.replace('B', '')
        driver = driver[12:]
        df2['driverID'] = int(driver)
        
        
    # calc_time
        df2['calc_time'] = calc_time[count]


    # dist gap
        distgap = (df1['sum_len'] - df1['flink_len'])/10000
        dist = []
        for i in range(len(df1['lon'])):
            if(i < len(df1['lon']) - 1):
                dist.append(haversine((df1.loc[i,'lon'], df1.loc[i, 'lat']), (df1.loc[(i+1), 'lon'], df1.loc[(i+1), 'lat'])))
        dist = pd.Series(dist)   
        for i in range(len(distgap) - 1):
            if(df1.loc[i, 'index'] == 0):
                link_start_point = i
            else:
                if(df1.loc[i, 'seq'] == df1.loc[link_start_point, 'seq']):
                    distgap[i] = distgap[i-1] +dist[i-1]
        
    # time gap
        hour = (df1['point_in_time'] / 10000).astype(int)
        minute = (df1['point_in_time'] / 100).astype(int) - hour * 100
        second = df1['point_in_time'].astype(int) - minute * 100 - hour * 10000
        timegap = hour * 60 * 60 + minute * 60 + second
        timegap = timegap - timegap[0]
        timegap = timegap.astype(float)
        df2['time_gap'] = timegap.values
        df2['dist_gap'] = distgap.values
        

        
    # states == 1.0
        states = [1.0] * len(df2['lngs'])
        df2['states'] = states
    
    # drop columns
        df2 = df2.drop(['Unnamed: 0', 'lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
             'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
             'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
             'ol_rot_spd', 'ol_spd', 'svc_name'])
        

        
    # append link_data ///without od_id and track travel time
        for i in range(1,len(df1.columns)):
            if df1.columns[i] == 'od_id' or df1.columns[i] == 'track_travel_time':
                continue
            df2[df1.columns[i]] = df1[df1.columns[i]].values
            
    # appned 4_summary values
        summary = df[df['od_id'] == IDs[count]]
        summary_Series = pd.Series(summary.values[0], index = summary.columns)
        df2 = df2.append(summary_Series)
        df2 = df2.drop('week')
        
    # 0.1m -> 1km, 0.1s -> 1s
        df2['flink_len'] = df2['flink_len'] / 10000
        df2['sum_len'] = df2['sum_len'] / 10000
        df2['track_speed'] = df2['track_speed'] / 10
        df2['track_sum_travel_time'] = df2['track_sum_travel_time'] / 10
        df2['od_len'] = df2['od_len']/10000
        df2['track_travel_time'] = df2['track_travel_time']/10
        df2['calc_travel_time'] = df2['calc_travel_time']/10
        
    #write file
        f.write(df2.to_json())
        f.write('\n')
        count = count + 1
    f.close()
    
    print('file was made')

In [13]:
import warnings

warnings.filterwarnings(action='ignore') 
#remove warning -> to see the result of the process

In [14]:
if __name__ =='__main__':
    df = pd.read_csv('./Data/4_summary_2.txt')
    IDs = df['od_id'].values   
    calc_time = df['calc_travel_time'].values
    
    divide_num = 5    ##  divide_num*20 = boganjeom
    
    folder_name = 'preprocessing2_'+str(divide_num*20)
    
    num_IDs = len(IDs)
    IDs_in_onefile = 1000
    num_file = int(num_IDs / IDs_in_onefile)+1

    procs=[]
    index = []
    for i in range(0,num_file):
        if IDs_in_onefile*(i+1) <=num_IDs:
            index = index+[[i*IDs_in_onefile, IDs_in_onefile*(i+1)]]
        else : 
            index = index+[[i*IDs_in_onefile, num_IDs]]
    for i, number in enumerate(index): #start 13 threads
        proc = Process(target=Make_preprocess, args=(folder_name, number[0], number[1], 
                                                     i, IDs, calc_time, divide_num))
        procs.append(proc)
        proc.start()


     Unnamed: 0         lon        lat  index  point_in_time  seq    flink  \
1            64  127.109083  37.474083      1         105858    0  2027630   
5            68  127.108611  37.474056      3         105904    1  2027633   
10           73  127.107861  37.473778      4         105945    2  2027627   
15           78  127.107083  37.473361      4         105959    3  2027658   
17           80  127.106833  37.473250      6         110006    3  2027658   
22           85  127.106639  37.473972      4         110042    4  2027655   
27           90  127.106389  37.474889      9         110128    4  2027655   
31           94  127.106194  37.475583     13         110205    4  2027655   
34           97  127.106167  37.475806      2         110207    5  2022194   
39          102  127.106000  37.476528      4         110216    6  2022197   
44          107  127.105833  37.477444      9         110226    6  2022197   
49          112  127.105639  37.478361     14         110237    

[121 rows x 30 columns]
Unnamed: 0                              64
lon                                127.109
lat                                37.4741
index                                    1
point_in_time                       105858
seq                                      0
flink                              2027630
tlink                              2027633
turn                                     3
flink_len                              140
sum_len                                140
facil                                    0
road_type                                9
link_type                                4
light                                    0
lane                                     2
outlink_cnt                              1
zone                              11680700
track_in_stamp                  1555811936
track_in_date                     20190421
track_in_time                       105856
track_speed                            229
track_travel_time             

Process Process-1:
Traceback (most recent call last):
  File "/home/centerdl/.local/lib/python3.5/site-packages/pandas/core/indexes/base.py", line 2657, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 987, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 993, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-f68a73bcc812>", line 91, in Make_preprocess
    dist.